In [85]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# Developed for Google LLC by RedCastle Resources Inc
# https://www.redcastleresources.com/

# <img width=50px  src = 'https://apps.fs.usda.gov/lcms-viewer/images/lcms-icon.png'> Lab 7: LCMS Map Validation

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/7-Map_Validation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/7-Map_Validation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/7-Map_Validation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## 7.0: Overview and Introduction


This notebook teaches how to assess map accuracy of LCMS outputs.

While this notebook teaches the specific steps LCMS takes to assess map accuracy, your assessment methods can possibly be much simpler if you have a large training sample, simple or systematic random sample design, and/or a map output that always takes the most confident prediction from the model. 

### 7.0.1: Background

* While previous tutorials have provided a good depiction of the model error, for a number of reasons, this is not a statistically valid depiction of the map error. The following are reasons why the model error is not the same as the map error in the case of LCMS' current methods:
    * Our sample is not a simple random sample - rather it is a stratified random sample
    * For Change, we introduce additional logic that makes the default model output different than the map output
* LCMS also lacks sufficient training data to leave a subset out of the actual model calibration
    * This means we need to simulate the model error using either a bootstrapping technique or k-fold cross validation
* If your particular project has sufficient training data to leave a subset out of the actual model calibration, it is a simple or systematic random sample (not stratified random), and the default model predicted class is the same as your final map class, you can skip this step and use the model errors from the methods used in Module 5.

### 7.0.2: Objective

In this tutorial, you learn how to assess the map accuracy of LCMS map outputs. 

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`
- `Google Cloud Storage`

The steps performed include:

- Understanding the difference between model and map accuracy
- Simulating map accuracy with k-fold cross validation

### 7.0.3: Before you begin

#### If you are working in Workbench: Set your current URL under `workbench_url`
This gives the Map Viewer a url in which to host the viewer we will be generating. 
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (See the image below)

![workspace url](img/workspace-url.png)

#### Set a folder to use for all exports under `export_path_root` 
* This folder should be an assets folder in an existing GEE project.
* By default, this folder is the same as the pre-baked folder (where outputs have already been created). 
* If you would like to create your own outputs, specify a different path for `export_path_root`, but leave the `pre_baked_path_root` as it was. This way, the pre-baked outputs can be shown at the end, instead of waiting for all exports to finish.
  * It will be something like `projects/projectID/assets/newFolder`
* This folder does not have to already exist. If it does not exist, it will be created

**If you are working in Qwiklabs and wish to export:** Copy the project ID from the 'Start Lab' screen into the `projectID` field in `export_path_root`.

In [49]:
workbench_url = 'https://1307eb830a12e633-dot-us-central1.notebooks.googleusercontent.com/lab/tree/lcms-training/7-Map_Validation.ipynb'
pre_baked_path_root  = 'projects/rcr-gee/assets/lcms-training'
export_path_root = pre_baked_path_root

# to export to a custom project, fill in your project ID and uncomment the lines below
# projectID = 'qwiklabs-gcp-00-00000000' # change this to match your project id
# export_path_root = f'projects/{projectID}/assets/lcms-training'

print('Export path root:', export_path_root)
print('Done')

Export path root: projects/rcr-gee/assets/lcms-training
Done


#### Installation
First, install necessary Python packages. Uncomment the first line to upgrade geeViz if necessary.

Note that we're also importing additional custom scripts from the lcms_scripts folder in this repo. 

In [50]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p
import inspect,operator,os,glob,json,warnings
import matplotlib.pyplot as plt
import pandas as pd  
import numpy as np

import lcms_scripts.accuracy_and_sampling_lib2 as asl
from importlib import reload

try:
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GroupKFold
    from sklearn.metrics import accuracy_score,classification_report,balanced_accuracy_score,cohen_kappa_score
    from sklearn import metrics 
except:
    !pip install -U scikit-learn
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GroupKFold
    from sklearn.metrics import accuracy_score,classification_report,balanced_accuracy_score,cohen_kappa_score
    from sklearn import metrics 

ee = getImagesLib.ee
Map = getImagesLib.Map

# Can set the port used for viewing map outputs
Map.port = 1235
print('Done')


Done


#### Set up your work environment

Create a folder in your export path where you will export the composites. In addition, create a blank image collection where your composites will live.

Currently, when running within Colab or Workbench, geeView uses a different project to authenticate through, so you may need to make your asset public to view from within Colab.

**Warning!!**

* **Read-only access is provided for all authenticated GEE users for pre-baked outputs**
* **If you are using the pre-baked output location (`export_path_root = pre_baked_path_root`), you will see errors for any operation that tries to write, delete, or change access permissions to any asset in the pre-baked output location.**
* **This is expected and will not stop you from successfully running this notebook - ignore these error messages should they appear.**


In [51]:
# Bring in all folders/collections that are needed
# These must already exist as they are created in previous notebooks

export_timeSync_folder = f'{export_path_root}/lcms-training_module-4_timeSync'

export_assembledLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-6_assembledLCMSOutputs'

# This is the pre-made TimeSync data
# Creating this dataset is not covered in this set of notebooks
timeSync_featureCollection = 'projects/lcms-292214/assets/R8/PR_USVI/TimeSync/18_PRVI_AllPlots_TimeSync_Annualized_Table_secLC'

# The model options table (created in module 5.1, but stored in the lcms-training repository)
model_options_csv_filename = './tables/LCMS_model_options_table.csv'
change_thresholds_json_filename = './tables/LCMS_change_thresholds.json'

# Set up folder to put accuracy files
local_map_acc_folder = '/tmp/lcms-training/local_map_accuracy'

if not os.path.exists(local_map_acc_folder):os.makedirs(local_map_acc_folder)

print('Done')

Done


In [52]:
# set up map
Map.clearMap()

# reset port if necessary
Map.port = 1237
Map.proxy_url = workbench_url

print('Done')

Done


## 7.1: Prepare input data



### 7.1.2: Bring in Reference Data

* First, we'll need to repeat steps from Module 5 and download our reference data to a local location

In [19]:
# Bring in raw TS data
timeSyncData = ee.FeatureCollection(timeSync_featureCollection)
timeSync_fields = timeSyncData.first().toDictionary().keys().getInfo()
# Now lets bring in all training data and prep it for modeling
assets = ee.data.listAssets({'parent': export_timeSync_folder})['assets']

# You may need to change the permissions for viewing model outputs in geeViz
# Uncomment this if needed
# for asset in assets:aml.updateACL(asset['name'],writers = [],all_users_can_read = True,readers = [])

# Read in each year of extracted TimsSync data
training_data = ee.FeatureCollection([ee.FeatureCollection(asset['name']) for asset in assets]).flatten()

# Bring in existing LCMS data for the class names, numbers, and colors
lcms_viz_dict = ee.ImageCollection("USFS/GTAC/LCMS/v2022-8").filter('study_area=="PRUSVI"').first().toDictionary().getInfo()

                                             
print('LCMS class code, names, and colors:',lcms_viz_dict)

# Specify the field for each product
reference_field_dict = {'Land_Cover':{'field':'DOM_SEC_LC'},
                        'Land_Use':{'field':'DOM_LU'},
                        'Change':{'field':'CP'}
                       }

# Get the field names for prediction
# Find any field that was not in the original TimeSync data and assume that is a predictor variable
all_fields = training_data.first().toDictionary().keys().getInfo()
predictor_field_names = [field for field in all_fields if field not in timeSync_fields]

# Filter out any non null values (any training plot with missing predictor data will cause the model to fail entirely)
training_data = training_data.filter(ee.Filter.notNull(predictor_field_names))

print('Done')

LCMS class code, names, and colors: {'Change_class_names': ['Stable', 'Slow Loss', 'Fast Loss', 'Gain', 'Non-Processing Area Mask'], 'Change_class_palette': ['3d4551', 'f39268', 'd54309', '00a398', '1b1716'], 'Change_class_values': [1, 2, 3, 4, 5], 'Land_Cover_class_names': ['Trees', 'Tall Shrubs & Trees Mix (SEAK Only)', 'Shrubs & Trees Mix', 'Grass/Forb/Herb & Trees Mix', 'Barren & Trees Mix', 'Tall Shrubs (SEAK Only)', 'Shrubs', 'Grass/Forb/Herb & Shrubs Mix', 'Barren & Shrubs Mix', 'Grass/Forb/Herb', 'Barren & Grass/Forb/Herb Mix', 'Barren or Impervious', 'Snow or Ice', 'Water', 'Non-Processing Area Mask'], 'Land_Cover_class_palette': ['005e00', '008000', '00cc00', 'b3ff1a', '99ff99', 'b30088', 'e68a00', 'ffad33', 'ffe0b3', 'ffff00', 'aa7700', 'd3bf9b', 'ffffff', '4780f3', '1b1716'], 'Land_Cover_class_values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'Land_Use_class_names': ['Agriculture', 'Developed', 'Forest', 'Non-Forest Wetland', 'Other', 'Rangeland or Pasture', 'No

In [20]:
# Now will download the training table to a local location

local_model_data_folder = '/tmp/lcms-training/local_modeling'
local_training_csv = os.path.join(local_model_data_folder,'timeSync_training_table.csv')


if not os.path.exists(local_model_data_folder):os.makedirs(local_model_data_folder)

# Download the training data from a featureCollection to a local CSV
# This function will automatically break the featureCollection into 5000 feature featureCollections
# if it is larger than the 5000 feature limit set by GEE
g2p.featureCollection_to_csv(training_data,local_training_csv,overwrite = False)

# Once the table is store locally, read it in
training_df = pd.read_csv(local_training_csv)

print('Done')

training_df.describe()

/tmp/lcms-training/local_modeling\timeSync_training_table.csv  already exists
Done


,BARREN,BARREN-GRA,BARREN-IMP,BARREN-SHR,BARREN-TRE,BARREN-TS,CHANGE_DUR,CP_Code,DOM_LU_Code,DOM_SEC_LC_Code,...,swir2_LT_dur,swir2_LT_fitted,swir2_LT_mag,swir2_LT_slope,wetness_LT_diff,wetness_LT_dur,wetness_LT_fitted,wetness_LT_mag,wetness_LT_slope,year
count,19777.0,19777.000000,19777.000000,19777.000000,19777.000000,19777.0,19777.000000,19777.000000,19777.000000,19777.000000,...,19777.000000,19777.000000,19777.000000,19777.000000,19777.000000,19777.000000,19777.000000,19777.000000,19777.000000,19777.000000
mean,0.0,0.005360,0.098094,0.004955,0.002174,0.0,14.327350,1.459372,3.307933,4.352885,...,26.316428,0.079638,0.005554,0.000505,-0.013641,29.233554,-0.073240,-0.020893,-0.000419,2011.236494
std,0.0,0.073016,0.297449,0.070221,0.046579,0.0,6.767562,1.035758,1.297727,4.435511,...,11.269293,0.039772,0.033831,0.005742,0.029336,10.805780,0.044394,0.042006,0.005689,6.040297
min,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.006793,-0.210283,-0.070094,-0.322420,1.000000,-0.440766,-0.483630,-0.053737,2000.517822
25%,0.0,0.000000,0.000000,0.000000,0.000000,0.0,9.000000,1.000000,3.000000,1.000000,...,17.000000,0.053389,-0.011057,-0.000472,-0.025358,21.000000,-0.096936,-0.039508,-0.001391,2006.169922
50%,0.0,0.000000,0.000000,0.000000,0.000000,0.0,17.000000,1.000000,3.000000,1.000000,...,28.000000,0.069914,0.006531,0.000233,-0.012719,37.000000,-0.066462,-0.021964,-0.000678,2011.295898
75%,0.0,0.000000,0.000000,0.000000,0.000000,0.0,21.000000,1.000000,3.000000,9.000000,...,38.000000,0.095523,0.021770,0.000901,0.000000,38.000000,-0.045340,0.000000,0.000000,2016.476685
max,0.0,1.000000,1.000000,1.000000,1.000000,0.0,21.000000,4.000000,6.000000,14.000000,...,38.000000,0.373654,0.300531,0.204653,0.381094,38.000000,0.072465,0.381094,0.226203,2021.411011


#### 7.2.1.1 Filter to predictors used

* Filter out to only have rows from the non correlated top 30 predictors
* No single set of prectors works best over all models and all model performance metrics
* Any subset of predictors could be used here, but this one should work well
* Good options are: `Non-correlated Predictors`, `All Predictors Top 30`, or `Non-correlated Predictors Top 30`


In [21]:
predictor_set = 'Non-correlated Predictors Top 30'
model_options = pd.read_csv(model_options_csv_filename)

model_options = model_options[model_options['Model Name'] == predictor_set]

display(model_options)

print('Done')

,Product Name,Model Name,OOB Acc,Overall Acc,Balanced Acc,Kappa,Var Imp
3,Change,Non-correlated Predictors Top 30,0.884917,0.846736,0.281699,0.094835,"['swir2_LT_slope', 'swir2_LT_fitted', 'NDVI_LT..."
7,Land_Cover,Non-correlated Predictors Top 30,0.974010,0.696878,0.286296,0.484086,"['red_CCDC_fitted', 'red_LT_fitted', 'NBR_LT_f..."
11,Land_Use,Non-correlated Predictors Top 30,0.993578,0.787196,0.626276,0.659068,"['red_LT_fitted', 'green_CCDC_fitted', 'red_CC..."


Done


## 7.2: Prepare validation framework
### 7.2.1: K-Fold Cross-validation

* LCMS does not have enough training samples to simply ommit 20% or so from training our final models
* Since our assemblage process introduces differences between the model predicted class, and our sample is based on a stratified random sample design, we cannot simply use the out-of-bag samples from the random forest model
* We have to use a method that will simulate the map accuracy that can account for the likelihood of each samples inclusion (strata weights), as well as also allow us to introduce any assemblage rules that are not typically part of the underlying random forest model
* Our methods roughly follow guidance from [Stehman 2014](./lit/Stehman_2014.pdf)
* We first divide our reference sample into a train and test set over K-folds and train a random forest model for each fold
* E.g. if we have 5 folds, then for each fold, 80% of the training data are used to train the model, and the remaining 20% are held out for comparing the reference and predicted classes. This ensures all samples are held out once ($20 \%  * 5 folds = 100\%$).
![Cross_Validation](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [22]:
# Set up k-fold cross-validation

products = ['Change','Land_Cover','Land_Use']
KFoldInfo = {}
# kfoldinfo_pickle_filename = pickleName+'.p'
# KFoldInfo['TrainingData'] = training_df.copy()

# strata = allTrainingData[stratColumn].squeeze()
groups = training_df['PLOTID'].squeeze()
k = 10
n_jobs = 4
gkf = GroupKFold(n_splits=k)
foldNum = 1
seed = 999
nTrees = 50

# Fit and Train model
# Set up a random forest model
rf = RandomForestClassifier(n_estimators = nTrees, random_state=seed,oob_score=False,n_jobs = n_jobs)

# Set up k fold in
KFoldInfo['STRATUM'] = []
KFoldInfo['STRATUM_PIXEL_COUNT'] = []
KFoldInfo['STRATUM_PIXEL_PCT'] = []

# Iterate over k folds
for train_index, test_index in gkf.split(training_df, training_df, groups):
    KFoldInfo[foldNum] = {}
    
    print(f'Fold {foldNum} has {len(train_index)} training samples and {len(test_index)} test samples')

    # Indices of training and test samples
    KFoldInfo[foldNum]['Indices'] = {\
        'Train': train_index,
        'Test': test_index}

    # Run model and predict probabilities
    KFoldInfo[foldNum]['Probabilities'] = {}
    KFoldInfo[foldNum]['Predictions'] = {}
    KFoldInfo[foldNum]['Model'] = {}
    KFoldInfo[foldNum]['Ref'] = {}
    
    # Pull the tran and test plots
    k_train,k_test = training_df.iloc[train_index], training_df.iloc[test_index]
    
    # Get the strata info
    KFoldInfo['STRATUM'].extend(k_test['STRATUM'])
    KFoldInfo['STRATUM_PIXEL_COUNT'].extend(k_test['STRATUM_PIXEL_COUNT'])
    KFoldInfo['STRATUM_PIXEL_PCT'].extend(k_test['STRATUM_PIXEL_PCT'])
    for product_name in products:
        print(f'Fitting model for fold {foldNum}, {product_name}')
        
        # Pull predictors from table from 5.1
        # Some parsing is needed to read it in properly
        predictor_variable_names = model_options[model_options['Product Name'] == product_name]['Var Imp'].values[0]
        predictor_variable_names = predictor_variable_names[1:-1]
        predictor_variable_names=predictor_variable_names.replace("'","").split(', ')
        
        # Get X and Y points for each group  
        kx_train = k_train[predictor_variable_names]
        ky_train = k_train[reference_field_dict[product_name]['field']+'_Code']
        
        kx_test = k_test[predictor_variable_names]
        ky_test = k_test[reference_field_dict[product_name]['field']+'_Code']
        
        rf.fit(kx_train,ky_train)
        
        # Get predicted classes or probabilities for each Test Point
        # For change, get the probabilities, for all others, get the classes
        if product_name in ['Land_Cover','Land_Use']:
            ky_pred = rf.predict(kx_test)
            
        else:
            ky_pred = rf.predict_proba(kx_test)
        KFoldInfo[foldNum]['Predictions'][product_name] = ky_pred
        KFoldInfo[foldNum]['Ref'][product_name] = ky_test
        
    foldNum+=1

print('Done')

Fold 1 has 17797 training samples and 1980 test samples
Fitting model for fold 1, Change
Fitting model for fold 1, Land_Cover
Fitting model for fold 1, Land_Use
Fold 2 has 17797 training samples and 1980 test samples
Fitting model for fold 2, Change
Fitting model for fold 2, Land_Cover
Fitting model for fold 2, Land_Use
Fold 3 has 17805 training samples and 1972 test samples
Fitting model for fold 3, Change
Fitting model for fold 3, Land_Cover
Fitting model for fold 3, Land_Use
Fold 4 has 17805 training samples and 1972 test samples
Fitting model for fold 4, Change
Fitting model for fold 4, Land_Cover
Fitting model for fold 4, Land_Use
Fold 5 has 17797 training samples and 1980 test samples
Fitting model for fold 5, Change
Fitting model for fold 5, Land_Cover
Fitting model for fold 5, Land_Use
Fold 6 has 17798 training samples and 1979 test samples
Fitting model for fold 6, Change
Fitting model for fold 6, Land_Cover
Fitting model for fold 6, Land_Use
Fold 7 has 17798 training samples 

### 7.2.2: Organize Reference and Predicted Values

* Organize reference and predicted values into simple dictionary by product

In [23]:
ref_pred_dict = {}

for product_name in products:
    preds = []
    refs = []
    for foldNum in range(1,k+1):
        predsFold = KFoldInfo[foldNum]['Predictions'][product_name]
        refsFold = KFoldInfo[foldNum]['Ref'][product_name]
        preds.extend(predsFold)
        refs.extend(refsFold)
        
    refs = pd.Series(refs)
    preds = pd.Series(preds)
    ref_pred_dict[product_name] = {'refs':refs,'preds':preds}
   
print(ref_pred_dict)

{'Change': {'refs': 0        1
1        1
2        1
3        1
4        4
        ..
19772    1
19773    1
19774    4
19775    1
19776    1
Length: 19777, dtype: int64, 'preds': 0          [0.98, 0.0, 0.0, 0.02]
1        [0.66, 0.02, 0.14, 0.18]
2         [0.94, 0.0, 0.04, 0.02]
3            [0.7, 0.0, 0.1, 0.2]
4          [0.84, 0.0, 0.1, 0.06]
                   ...           
19772      [0.54, 0.0, 0.1, 0.36]
19773        [0.8, 0.0, 0.1, 0.1]
19774     [0.82, 0.0, 0.06, 0.12]
19775     [0.88, 0.0, 0.04, 0.08]
19776     [0.78, 0.0, 0.08, 0.14]
Length: 19777, dtype: object}, 'Land_Cover': {'refs': 0        10
1        10
2        10
3        10
4        10
         ..
19772     1
19773     1
19774     7
19775     8
19776     8
Length: 19777, dtype: int64, 'preds': 0        10
1         1
2        10
3        10
4        10
         ..
19772     1
19773     1
19774     8
19775     1
19776    10
Length: 19777, dtype: int64}, 'Land_Use': {'refs': 0        4
1        4
2        1
3      

### 7.2.3: Apply Thresholds to Change

* Since the Change outputs use a specific set of thresholds and logic covered in Module 6, we must replicate this process to ensure the predicted Change classes reflect what would be on the map.
* There are many additional considerations for representing time series map accuracy
    * Often, since our input predictor data may not indicate a change occurred until the year after, we will allow for a prediction the year after to count as correct. These kind of fuzzy logic exceptions are not included in this example, but may be appropriate to consider. 

In [24]:
product_name = 'Change'

# Open class thresholds used from Module 6
o = open(change_thresholds_json_filename,'r')
change_thresholds = json.load(o)
o.close()

# Pull the predictions from the k-fold cross validation
# Currently they contain the probability of all predictions
raw_change_preds = np.array(ref_pred_dict[product_name]['preds'].values.tolist())
print('Input Change Predictions:',raw_change_preds)

# Specify which classes to assemble and the index (value - 1) where they are located in the prediction
# probability table
change_classes = ['Slow Loss','Fast Loss','Gain']
change_indices = [1,2,3]

# Find the max probability
max_prob = np.max(raw_change_preds, axis=1)
print(max_prob)

# Set up a constant output of 1 (Stable)
out = np.ones(len(max_prob))

# Find max conf class
change_class_max_conf_preds = raw_change_preds.argmax(axis=1)+1

# Iterate across each class 
for change_class,change_index in list(zip(change_classes,change_indices)):
    
    # Pull the predictions for the class
    preds_class = raw_change_preds[:,change_index]

    # Find if it is the max and above the threshold
    isMax = preds_class == max_prob
    aboveThresh = preds_class > change_thresholds[change_class]
    isChange = isMax & aboveThresh

    # If it is the max and above the threshold, recode the stable to the respective value (index +1)
    change_value = change_index+1
    out[isChange] = change_value

print('Output Change Predictions:',out)
# Reset the predictions for Change
ref_pred_dict[product_name]['preds'] = pd.Series(out)

print('Done')

Input Change Predictions: [[0.98 0.   0.   0.02]
 [0.66 0.02 0.14 0.18]
 [0.94 0.   0.04 0.02]
 ...
 [0.82 0.   0.06 0.12]
 [0.88 0.   0.04 0.08]
 [0.78 0.   0.08 0.14]]
[0.98 0.66 0.94 ... 0.82 0.88 0.78]
Output Change Predictions: [1. 1. 1. ... 1. 1. 1.]
Done


## 7.3 Compute Accuracy

### 7.3.1: Set up strata and strata weights

* In order to account for the probability of including a sample from a given stratum, we need to know the proportion of the total sample location population for each stratum 
* Note that our strata are not the same as our map classes - we set the strata before we make the map. the goal of  the strata is to make sure we appropriately sample classes that are underrepresented.
* The number of pixels in each class will be used as weights in our accuracy assessmentWe can never expect a model-based output to match this level of detail.
The question becomes whether the maps are useful and do they at least align with the literature that employed similar methods.
Reviewing the maps, we can see they are a reasonable depiction of what they intend to represent, but should be used with these error rates in mind. They are generally appropriate to use over larger areas, but not for your favorite pixel.


![strata](img/lab-7/strata-PRUSVI.png)

Source: [LCMS Methods v2020-6](https://data.fs.usda.gov/geodata/rastergateway/LCMS/LCMS_v2020-6_Methods.pdf)

In [25]:
# Set up lookup dictionary to convert the numeric codes to names
strata_code_name_dict = ee.Dictionary({1:"DEVELOPED",
                                       2: "WATER",
                                       3: "BARREN",
                                       4: "AGRICULTURE",
                                       5: "NON-FORESTED WETLAND",
                                       6: "FORESTEED WETLAND",
                                       7: "RANGE-GRASS-SHRUB-HERB",
                                       8: "EVERGREEN",
                                       9: "DECIDUOUS",
                                       10: "CLOUD FOREST",
                                       11: "COASTAL MIXED FOREST"
                                      })

stratum = [i for i in KFoldInfo['STRATUM']]

stratum_counts = KFoldInfo['STRATUM_PIXEL_COUNT']
stratum_pct = [(n/100) for n in KFoldInfo['STRATUM_PIXEL_PCT']]
strata_dict = dict(set(zip(stratum,stratum_counts)))
strata_pct_dict = dict(set(zip(stratum,stratum_pct)))
strata_dict_sorted = sorted(strata_dict.items(), key = lambda x:x[0])
strata_pct_dict_sorted = sorted(strata_pct_dict.items(), key = lambda x:x[0])

print(strata_dict_sorted)

print('Done')

[(1, 1450331), (2, 113212), (3, 110101), (4, 344785), (5, 71185), (6, 92958), (7, 3519261), (8, 3325898), (9, 865643), (10, 258676), (11, 102288)]
Done


### 7.3.2: Look at confusion matrices

* We will first look at the confusion matrices from the cross-validation
* These will be very similar to the matrices shown in Module 5
* But, here we area assessing map error- rather than model error

* Notice the impact of the change assemblage (thresholds) is very minor. While the overall accuraccy is not changed much, the map is much different (as shown in module 6).

* Remember that we are not mapping slow loss in Puerto Rico since it is so uncommon in our training data

In [26]:
reload(asl)
# Ignore divide by 0 warnings from NumPy
warnings.filterwarnings('ignore')

# Iterate across each product
for product in products:
    product_title = product.replace('_',' ')

    # Set up output filename
    cm_file = os.path.join(local_map_acc_folder,f'LCMS_{product}_Map_Confusion_Matrix.csv')

    # Get the class numbers, names, and values 
    class_numbers = list(set(ref_pred_dict[product]['refs']))
    class_names = lcms_viz_dict[f'{product}_class_names'][:-1]
    class_values = lcms_viz_dict[f'{product}_class_values'][:-1]
    lookup_dict = dict(list(zip(class_values,class_names)))

    # Get rid of any missing values
    assessment_classes = [n for n in class_values if n  in class_numbers]
    labels = [lookup_dict[n] for n in class_values if n  in class_numbers]

    # Get the confusion matrix
    print(f'Final Assembled {product_title} Map Confusion Matrix:')
    cm = asl.getConfusionMatrix(ref_pred_dict[product]['refs'],ref_pred_dict[product]['preds'],stratum,stratum_pct,strata_dict,assessment_classes,labels)
    display(cm)
    cm.to_csv(cm_file)
    
    # Show the confusion matrix of change if we simply used the max model confidence (only slightly different)
    if product == 'Change':
        print('Change Max Conf:')
        cm = asl.getConfusionMatrix(ref_pred_dict[product]['refs'],change_class_max_conf_preds,stratum,stratum_pct,strata_dict,assessment_classes,labels)
        display(cm)

print('Done')

Final Assembled Change Map Confusion Matrix:


Observed            \
                                                     Stable Slow Loss   
Predicted Stable                                    2565.42      2.75   
          Slow Loss                                    0.00      0.00   
          Fast Loss                                    2.58      0.00   
          Gain                                        20.41      0.00   
          Producer's Accuracy (1 - omission error)    99.09      0.00   

                                                                      \
                                                   Fast Loss    Gain   
Predicted Stable                                      151.32  444.18   
          Slow Loss                                     0.00    0.00   
          Fast Loss                                     8.20    0.38   
          Gain                                          2.56   13.75   
          Producer's Accuracy (1 - omission error)      5.36    3.11   

                                                                                           
                                                   User's Accuracy (1 - commission error)  
Predicted Stable                                                                    81.85  
          Slow Loss                                                                   NaN  
          Fast Loss                                                                 74.54  
          Gain                                                                      36.65  
          Producer's Accuracy (1 - omission error)                                  81.30

Change Max Conf:


Observed            \
                                                     Stable Slow Loss   
Predicted Stable                                    2569.06      2.75   
          Slow Loss                                    0.00      0.00   
          Fast Loss                                    2.19      0.00   
          Gain                                        17.15      0.00   
          Producer's Accuracy (1 - omission error)    99.23      0.00   

                                                                      \
                                                   Fast Loss    Gain   
Predicted Stable                                      152.68  446.44   
          Slow Loss                                     0.00    0.00   
          Fast Loss                                     7.85    0.35   
          Gain                                          1.55   11.53   
          Producer's Accuracy (1 - omission error)      5.15    2.63   

                                                                                           
                                                   User's Accuracy (1 - commission error)  
Predicted Stable                                                                    81.79  
          Slow Loss                                                                   NaN  
          Fast Loss                                                                 77.14  
          Gain                                                                      37.23  
          Producer's Accuracy (1 - omission error)                                  81.34

Final Assembled Land Cover Map Confusion Matrix:


Observed  \
                                                      Trees   
Predicted Trees                                     1783.86   
          Shrubs & Trees Mix                           1.37   
          Grass/Forb/Herb & Trees Mix                 12.04   
          Barren & Trees Mix                           0.00   
          Shrubs                                       7.65   
          Grass/Forb/Herb & Shrubs Mix                 5.94   
          Barren & Shrubs Mix                          0.04   
          Grass/Forb/Herb                             46.26   
          Barren & Grass/Forb/Herb Mix                 0.00   
          Barren or Impervious                        15.45   
          Water                                        0.69   
          Producer's Accuracy (1 - omission error)    94.89   

                                                                       \
                                                   Shrubs & Trees Mix   
Predicted Trees                                                129.28   
          Shrubs & Trees Mix                                     0.01   
          Grass/Forb/Herb & Trees Mix                            5.10   
          Barren & Trees Mix                                     0.00   
          Shrubs                                                 0.34   
          Grass/Forb/Herb & Shrubs Mix                           1.03   
          Barren & Shrubs Mix                                    0.00   
          Grass/Forb/Herb                                       15.19   
          Barren & Grass/Forb/Herb Mix                           0.00   
          Barren or Impervious                                   0.00   
          Water                                                  0.00   
          Producer's Accuracy (1 - omission error)               0.06   

                                                                                \
                                                   Grass/Forb/Herb & Trees Mix   
Predicted Trees                                                         158.71   
          Shrubs & Trees Mix                                              1.34   
          Grass/Forb/Herb & Trees Mix                                    12.58   
          Barren & Trees Mix                                              0.00   
          Shrubs                                                         17.51   
          Grass/Forb/Herb & Shrubs Mix                                    9.77   
          Barren & Shrubs Mix                                             0.00   
          Grass/Forb/Herb                                                49.34   
          Barren & Grass/Forb/Herb Mix                                    0.00   
          Barren or Impervious                                           11.51   
          Water                                                           0.00   
          Producer's Accuracy (1 - omission error)                        4.52   

                                                                              \
                                                   Barren & Trees Mix Shrubs   
Predicted Trees                                                  4.14  46.62   
          Shrubs & Trees Mix                                     0.00   3.43   
          Grass/Forb/Herb & Trees Mix                            0.01   6.51   
          Barren & Trees Mix                                     0.00   0.00   
          Shrubs                                                 0.00  15.96   
          Grass/Forb/Herb & Shrubs Mix                           0.00  11.05   
          Barren & Shrubs Mix                                    0.05   0.00   
          Grass/Forb/Herb                                        0.50  25.07   
          Barren & Grass/Forb/Herb Mix                           0.00   0.00   
          Barren or Impervious                                   0.11   3.92   
          Water                     

Final Assembled Land Use Map Confusion Matrix:


Observed            \
                                                   Agriculture Developed   
Predicted Agriculture                                     4.08      0.85   
          Developed                                       7.76    480.82   
          Forest                                          5.47     61.71   
          Non-Forest Wetland                              0.13      0.41   
          Other                                           0.00      0.00   
          Rangeland or Pasture                           33.77     51.83   
          Producer's Accuracy (1 - omission error)        9.46     81.98   

                                                             \
                                                     Forest   
Predicted Agriculture                                  4.48   
          Developed                                   23.96   
          Forest                                    1757.20   
          Non-Forest Wetland                          11.20   
          Other                                        0.22   
          Rangeland or Pasture                        75.95   
          Producer's Accuracy (1 - omission error)    93.99   

                                                                              \
                                                   Non-Forest Wetland  Other   
Predicted Agriculture                                            0.00   0.00   
          Developed                                              1.47   3.32   
          Forest                                                11.51  16.80   
          Non-Forest Wetland                                    13.69   0.10   
          Other                                                  0.61   5.94   
          Rangeland or Pasture                                  18.93   0.00   
          Producer's Accuracy (1 - omission error)              35.48  57.93   

                                                                         \
                                                   Rangeland or Pasture   
Predicted Agriculture                                              7.90   
          Developed                                               33.41   
          Forest                                                 202.86   
          Non-Forest Wetland                                      15.78   
          Other                                                    0.06   
          Rangeland or Pasture                                   359.34   
          Producer's Accuracy (1 - omission error)                56.50   

                                                                                           
                                                   User's Accuracy (1 - commission error)  
Predicted Agriculture                                                               34.60  
          Developed                                                                 85.83  
          Forest                                                                    85.60  
          Non-Forest Wetland                                                        41.83  
          Other                                                                     94.13  
          Rangeland or Pasture                                                      64.79  
          Producer's Accuracy (1 - omission error)                                  81.57

Done


### 7.3.3: Run Stehman 2014 Accuracy Method

* Once the reference and predicted classes are organized for each LCMS output, we can run a method that reflects the method outlined in [Stehman 2014](https://www.tandfonline.com/doi/abs/10.1080/01431161.2014.930207) for each LCMS output product.
* The outputs are stored as `.csv` files 

* The documentation of the function used is shown [here](https://github.com/redcastle-resources/lcms-training/blob/048c6c19b33b793a2ff0ef07fe212038acba87d7/lcms_scripts/accuracy_and_sampling_lib2.py#L343)

In [40]:

reload(asl)
acc_results = {}
for product_name in products:
    print(f'Computing accuracy for: {product_name}')
    acc_file = os.path.join(local_map_acc_folder,f'LCMS_{product_name}_Map_Accuracy_Results.csv')
    
    accuracy, balanced_accuracy, users, producers, kappa, f1_score, areas, accuracy_error, usersError, producersError, area_errors = asl.get_write_stratified_accuracies(\
        ref_pred_dict[product_name]['refs'],            # The correct classifications
        ref_pred_dict[product_name]['preds'],      # The predicted classifications
        stratum,       # The strata of the same plots as above
        strata_dict,         # Dictionary of the number of pixels in each stratum - defined in LCMSVariables - used for weighting
        lcms_viz_dict[f'{product_name}_class_values'][:-1], # Class names - used for looping through classes for users/producers accuracies and areas
        lcms_viz_dict[f'{product_name}_class_names'][:-1],
        method = product_name,        # This is just a run name, used for printing out accuracies in file. Not really used anymore
        accFile = acc_file)

    acc_results[product_name] = accuracy, balanced_accuracy, users, producers, kappa, f1_score, areas, accuracy_error, usersError, producersError, area_errors 

print('Done')

Computing accuracy for: Change
Computing accuracy for: Land_Cover
Computing accuracy for: Land_Use
Done


### 7.3.4: Look at Accuracy Outputs

* Once finished, we can look at the results

In [42]:
# Get each output .txt file and open it 
results_files = glob.glob(os.path.join(local_map_acc_folder,'*Accuracy*csv'))
for results_file in results_files:
    results = pd.read_csv(results_file)
    display(results)

,Metric,Value,Confidence
0,Accuracy,0.813018,0.003347
1,Balanced Accuracy,0.286327,NaN
2,Kappa,0.053835,NaN
3,F1 Score,0.813018,NaN
4,Users Accuracy: Stable,0.818520,0.003391
5,Users Accuracy: Slow Loss,NaN,0.000000
6,Users Accuracy: Fast Loss,0.745445,0.000427
7,Users Accuracy: Gain,0.366489,0.000553
8,Producers Accuracy: Stable,0.990908,0.003391
9,Producers Accuracy: Slow Loss,0.000000,0.000000


,Metric,Value,Confidence
0,Accuracy,0.699712,0.003792
1,Balanced Accuracy,0.213822,NaN
2,Kappa,0.342598,NaN
3,F1 Score,0.699712,NaN
4,Users Accuracy: Trees,0.769289,0.003822
5,Users Accuracy: Tall Shrubs & Trees Mix (SEAK ...,NaN,0.000000
6,Users Accuracy: Shrubs & Trees Mix,0.015427,0.000029
7,Users Accuracy: Grass/Forb/Herb & Trees Mix,0.157391,0.000547
8,Users Accuracy: Barren & Trees Mix,0.000000,0.000000
9,Users Accuracy: Tall Shrubs (SEAK Only),NaN,0.000000


,Metric,Value,Confidence
0,Accuracy,0.815691,0.003217
1,Balanced Accuracy,0.487836,NaN
2,Kappa,0.597439,NaN
3,F1 Score,0.815691,NaN
4,Users Accuracy: Agriculture,0.345983,0.000295
5,Users Accuracy: Developed,0.858257,0.002596
6,Users Accuracy: Forest,0.855959,0.003682
7,Users Accuracy: Non-Forest Wetland,0.418267,0.000485
8,Users Accuracy: Other,0.941289,0.000372
9,Users Accuracy: Rangeland or Pasture,0.647914,0.002585


### 7.3.5: Interpret results
* Notice the accuracies are low for many classes - especially change classes
* Stable generally has high accuracy, and skews the overall accuracy, but the balanced accuracy and the user's and producer's accuracy are low. 
* While it is true there is a lot of disagreement between our TimeSync-based reference data and our maps, this has been shown to be the reality throughout the literature
    * E.g. 
        * [Cohen et al 2018](./lit/Cohen_et_al_2018_Landtrendr_Ensemble.pdf)
            * Tested various LandTrendr band outputs for detecting change. Omission error rates were between ~0.45 and ~0.60 and commission error rates were > 0.825.
        * [Healey et al 2018](./lit/Healey_et_al_2018_RSE_LCMS_Approach.pdf)
            * Tested various change algorithms for predicting change much like LCMS does now. The best output had a balanced omission and commission error rate of 0.4.
        * [Stehman et al 2021](./lit/Stehman_et_al_2021_LCMAP_Acc.pdf)
            * Used an independent TimeSync-based reference dataset to validate USGS Land Change Monitoring Assessment and Projection (LCMAP) outputs. The change (any spectral break by CCDC - covered in Module 3), had a commission error rate of 0.86 and omission error rate of 0.83 (Table 8).
    * Each of these papers relied on TimeSync-based reference data that had analysts include every piece of information available through the use of tools such as Google Earth's high resolution time lapse, other additional high resolution aerial photography, ancillary datasets such as the Monitoring Trends in Burn Serverity (MTBS), etc.
    * We can never expect a model-based output to match this level of detail.
    * The question becomes whether the maps are useful and do they at least align with the literature that employed similar methods.
    * Reviewing the maps, we can see they are a reasonable depiction of what they intend to represent, but should be used with these error rates in mind. They are generally appropriate to use over larger areas, but not for your favorite pixel.
    * Additional considerations is often with time series mapping, correct classifications will be allowed if the model predicted the class in the exact year or an adjacent year. The method outlined in this module did not employ such methods.


## 7.4 Copy outputs to Google Cloud Storage
### 7.4.1 Set up Google Cloud Storage Bucket

* This step will set up a location in Google Cloud Storage for the accuracy results `.txt` files to be copied to
* You can set up your own export location by specifying a unique name for the bucket


In [43]:
pre_baked_accuracy_output_bucket = f'gs://lcms-training-outputs-accuracy'

projectID = 'qwiklabs-gcp-00-00000000' # change this to match your project id
personal_accuracy_output_bucket = f'{projectID}-outputs-accuracy'

# set accuracy output bucket 
accuracy_output_bucket = pre_baked_accuracy_output_bucket

# You may need to run these functions if running outside Workbench
# In order to avoid permissions issues, ensure the projectName is the same as the one you used for authenticating 
# to GEE
# !gcloud auth login
# !gcloud projects list
# !gcloud config set project projectName

!gsutil mb {accuracy_output_bucket}

Creating gs://lcms-training-outputs-accuracy/...
ServiceException: 409 A Cloud Storage bucket named 'lcms-training-outputs-accuracy' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### 7.4.2: Copy outputs to Google Cloud Storage

* Copy outputs to a bucket. 

In [47]:
!gsutil -m cp {local_map_acc_folder}/*.csv {accuracy_output_bucket}

Copying file://\tmp\lcms-training\local_map_accuracy\LCMS_Land_Cover_Map_Confusion_Matrix.csv [Content-Type=text/csv]...
/ [0/6 files][    0.0 B/  8.0 KiB]   0% Done                                    
Copying file://\tmp\lcms-training\local_map_accuracy\LCMS_Land_Use_Map_Confusion_Matrix.csv [Content-Type=text/csv]...
/ [0/6 files][    0.0 B/  8.0 KiB]   0% Done                                    
Copying file://\tmp\lcms-training\local_map_accuracy\LCMS_Land_Use_Map_Accuracy_Results.csv [Content-Type=text/csv]...
/ [0/6 files][    0.0 B/  8.0 KiB]   0% Done                                    
Copying file://\tmp\lcms-training\local_map_accuracy\LCMS_Change_Map_Accuracy_Results.csv [Content-Type=text/csv]...
/ [0/6 files][    0.0 B/  8.0 KiB]   0% Done                                    
Copying file://\tmp\lcms-training\local_map_accuracy\LCMS_Land_Cover_Map_Accuracy_Results.csv [Content-Type=text/csv]...
/ [0/6 files][    0.0 B/  8.0 KiB]   0% Done                                   

In [100]:
# Open this link to view the accuracy outputs
print(f'https://console.cloud.google.com/storage/browser/{os.path.basename(accuracy_output_bucket)}')

https://console.cloud.google.com/storage/browser/lcms-training-outputs-accuracy


## Lab 7 challenge:

**If you are accessing this lab through Qwiklabs, this challenge will be assessed for completion in Lab 7.**
1. Change the thresholds used to find fast loss and gain and apply to the raw predicted Change probabilities

    * Use `{'Slow Loss':0,'Fast Loss': 0.05, 'Gain': 0.15}` as your thresholds

    * Use the `raw_change_preds` variable from earlier in the module as your raw change predictions

    * Use many of the same methods as used section 7.1.5
<br>

2. Create a confusion matrix of the resulting assembled predicted 

    * Use `ref_pred_dict['Change']['refs']` as your observed values

    * Use the `asl.getConfusionMatrix` function to get the confusion matrix
        
    * Take note the results change very very little from the results from section 7.2.1.
    <br> 
3. Save confusion matrix to a csv file.

   * Save csv to this path: `"/tmp/challenge/module_7_challenge_answer.csv"`
     * **Note: The path to the csv must exactly match the path above.** 
    <br>
    
    * Create the `"/tmp/challenge"` folder if it does not already exist.
      
        Example:
    ```python
        out_csv = "/tmp/challenge/module_7_challenge_answer.csv"
        if not os.path.exists(os.path.dirname(out_csv)):os.makedirs(os.path.dirname(out_csv))
    ```
<br>

4.  Check that the output csv exists.
    
    * Example: 
    ```python
        print(os.path.exists(out_csv))
    ```
<br>


In [48]:
# insert challenge code here



Observed            \
                                                     Stable Slow Loss   
Predicted Stable                                    2565.42      2.75   
          Slow Loss                                    0.00      0.00   
          Fast Loss                                    2.58      0.00   
          Gain                                        20.41      0.00   
          Producer's Accuracy (1 - omission error)    99.09      0.00   

                                                                      \
                                                   Fast Loss    Gain   
Predicted Stable                                      151.32  444.18   
          Slow Loss                                     0.00    0.00   
          Fast Loss                                     8.20    0.38   
          Gain                                          2.56   13.75   
          Producer's Accuracy (1 - omission error)      5.36    3.11   

                                                                                           
                                                   User's Accuracy (1 - commission error)  
Predicted Stable                                                                    81.85  
          Slow Loss                                                                   NaN  
          Fast Loss                                                                 74.54  
          Gain                                                                      36.65  
          Producer's Accuracy (1 - omission error)                                  81.30

True


## Done with Module 7